# 1. 크롤링과 함께 쓰는 mongoDB 예제

### cine21 인물 랭킹 알아내기
  - http://www.cine21.com/rank/person/
  - 조회를 눌렀을 때, Go to Network -> content 의 Request URL/Method 와 Form 데이터 알아내기
    - Request URL: http://www.cine21.com/rank/person/content
    - Request Method: POST
    - Form Data
      - section:actor
      - period_start:2021-10
      - gender:all
      - page:1
    - 하단부 페이지를 누를 때마다, Form Data 의 page 값이 바뀜

In [2]:
import requests as req
import re
import datetime
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
import pymongo

In [3]:
# 몽고DB 서버 연결
username = 'rapa00'
password = '1234'
conn = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))

In [4]:
# db, collection 연결(없으면 생성)
# python과 연동되었을 때, db와 collection의 경우, document insert 싯점에 생성됨
mongodb = conn.cine21
#collection 객체 변수 = 몽고db.컬렉션이름(없으면 만들어짐)
actor_collection = mongodb.actor_collection 

In [17]:
# actor 정보가 있는지 확인하기
actor_list = actor_collection.find()
for actor in actor_list:
    print(actor['actor'])

KeyError: 'actor'

In [6]:
# request url
cine21_url = 'http://www.cine21.com/rank/person/content'
month = "2021-10"
year = "2020-11"

# post 방식으로 데이터를 가져오기 위한 request 시 필요 정보
# 브라우저의 개발자 모드의 network tab 의 form 정보
conditions = dict()
conditions['section'] = 'actor'
conditions['period_start'] = month
conditions['gender'] = 'all'
conditions['page'] = 1

# post 방식으로 request 요청
response = req.post(cine21_url, data = conditions)

In [7]:
response

<Response [200]>

In [69]:
response.content

b'\t\t\t<ul class="people_list">\r\n\t\t\t\t<li class="people_li">\r\n\t\t\t\t\t<a href="/db/person/info/?person_id=74826"><img src="https://image.cine21.com/resize/cine21/person/2019/0828/11_54_52__5d65ecfc13ca6[X145,145].jpg" alt="" class="people_thumb" target="_blank" /></a>\r\n\t\t\t\t\t<div class="name"><a href="/db/person/info/?person_id=74826">\xeb\xb0\x95\xec\xa0\x95\xeb\xaf\xbc(1\xed\x8e\xb8)</a></div>\r\n\t\t\t\t\t<ul class="num_info">\r\n\t\t\t\t\t\t<li><span class="tit">\xed\x9d\xa5\xed\x96\x89\xec\xa7\x80\xec\x88\x98</span><strong>161,725</strong></li>\r\n\t\t\t\t\t\t<!--\r\n\t\t\t\t\t\t<li><a href="#" class="btn_graph"><span class="ico"></span><span>\xed\x9d\xa5\xed\x96\x89\xec\x84\xb1\xec\xa0\x81<br />\xea\xb7\xb8\xeb\x9e\x98\xed\x94\x84\xeb\xa1\x9c \xeb\xb3\xb4\xea\xb8\xb0</span></a></li>\r\n\t\t\t\t\t\t-->\r\n\t\t\t\t\t</ul>\r\n\t\t\t\t\t<!-- \xec\x98\x81\xed\x99\x94\xed\x8f\xac\xec\x8a\xa4\xed\x84\xb0\xeb\x8a\x94 \xec\xb5\x9c\xeb\x8c\x80 5\xea\xb0\x9c\xea\xb9\x8c\xec\

In [8]:
# html 문서인 soup 객체 생성
#soup = bs(response.content.decode('utf-8'), 'html.parser')
soup = bs(response.content, 'html.parser')
soup

 <ul class="people_list">
<li class="people_li">
<a href="/db/person/info/?person_id=74826"><img alt="" class="people_thumb" src="https://image.cine21.com/resize/cine21/person/2019/0828/11_54_52__5d65ecfc13ca6[X145,145].jpg" target="_blank"/></a>
<div class="name"><a href="/db/person/info/?person_id=74826">박정민(1편)</a></div>
<ul class="num_info">
<li><span class="tit">흥행지수</span><strong>161,725</strong></li>
<!--
						<li><a href="#" class="btn_graph"><span class="ico"></span><span>흥행성적<br />그래프로 보기</span></a></li>
						-->
</ul>
<!-- 영화포스터는 최대 5개까지만 -->
<ul class="mov_list">
<li>
<a href="/movie/info/?movie_id=57227">
<img alt="" class="thumb" src="https://image.cine21.com/resize/cine21/poster/2021/0928/10_31_26__6152706e344e6[X85,120].jpg" target="_blank"/>
<span>기적</span>
</a>
</li>
</ul>
<!-- 순위 --><span class="grade">1</span>
</li>
<li class="people_li">
<a href="/db/person/info/?person_id=25123"><img alt="" class="people_thumb" src="https://image.cine21.com/resize/cine21/person/

In [9]:
actors =  soup.select("li.people_li div.name")
actors
#<div class="name"><a href="/db/person/info/?person_id=74826">박정민(1편)</a></div>   
for actor in actors:
    #actor.text
    actor.getText(strip=True)
    print(actor.text)

박정민(1편)
이성민(1편)
임윤아(1편)
노회찬(1편)
이수경(1편)
박영남(1편)
김환진(1편)


In [10]:
# 정규 표현식 참고 : https://regexr.com/
# 문자, 숫자를 찾음 : \w  0-9a-zA-Z
# 특수 기호 () 문자로 인식 : \( \)

# 2. 반복 표현 ?, *, +
# + : 앞 문자가 1번 또는 그 이상 반복되는 패턴
# ? : 앞 문자가 0번 또는 1번 표시되는 패턴(없어도 되고, 한번 있어도 되는 패턴)
# * : 앞 문자가 0번 또는 그이상 반복되는 패턴
import re
test_data = '마동석(17편)'
#re.sub('정규표현식', '변경데이터', test_data)
print(re.sub("\(\w+\)", "", test_data))

마동석


In [11]:
# [실습]
# 한 페이지 내의 배우이름만 가져오기
import re

actors_list = []  # actors_list = list()
for actor in actors:
    #print(re.sub("\(\w+\)", "", actor.getText(strip=True))) 
    actors_list.append(re.sub("\(\w+\)", "", actor.getText(strip=True)))
actors_list   # 배우 리스트

['박정민', '이성민', '임윤아', '노회찬', '이수경', '박영남', '김환진']

### 각 배우별 상세 정보를 document에 넣고 싶다.
* 각 배우별 상세 정보를 별도 컬럼으로 만들려했더니, 각 배우별 상세 정보 항목이 다르다!
* 모든 상세 정보 항목을 컬럼으로 만들고, 각 컬럼에 매칭되는 컬럼값을 넣기가 쉽지 않다. 코드도 복잡하고!
* Mongodb는 NoSQL -> 통째로 집어넣자.!

* embedded document
  - document 의 컬럼값으로 document를 넣을 수 있다.

ul.people_list li.people_li    
<a href="/db/person/info/?person_id=74826">박정민(1편)</a>   
- url : 패턴 찾기  
박정민    http://www.cine21.com + /db/person/info/?person_id=74826   
         http://www.cine21.com/db/person/info/?person_id=74826   
윤아      http://www.cine21.com +  /db/person/info/?person_id=74826  
         http://www.cine21.com/db/person/info/?person_id=74826  
이성민    http://www.cine21.com +  /db/person/info/?person_id=25123  

In [12]:
# 배우 상세 정보 추출을 위한 과정 셀
actors =  soup.select("li.people_li div.name")
#actors
#actors[0].select_one('a').attrs['href']  # 한사람에 대한 href 속성의 값
#actors_detail_info = list() # 배우 상세 정보 저장
for actor in actors:
    # 배우 한명의 상세정보를 위한 url
    #print("http://www.cine21.com" + actor.select_one('a').attrs['href'])
    url_link = "http://www.cine21.com" + actor.select_one('a').attrs['href']

    res = req.get(url_link)

    # 배우 상세정보를 저장하는 soup 객체(html)
    soup_actor = bs(res.content, 'html.parser') 
    # 한 배우에 대한 정보 리스트
    #default_info = soup_actor.select_one('ul.default_info')
    #print(default_info)
    #actor_info_dict = dict() # 배우 한명 정보를 담는 dict 변수

    default_info = soup_actor.select_one('ul.default_info')
    #print(default_info)
    # 한 배우 상세정보를 리스트로 리턴받음    
    actor_details = default_info.select('li') 
    for actor_detail in actor_details:
        # 상세정보 key 추출
        #print(actor_detail.select_one('span.tit').get_text(strip=True))
        #print(type(actor_detail))
        # 상세정보 value 추출
        actor_detail_value = re.sub('<span.*?>.*?</span>','', str(actor_detail))
        actor_detail_value = re.sub('<.*?>','', actor_detail_value).strip()
        
        #배우 한 명의 정보 dict 저장
        print(actor_detail_value)    
    print("-" * 10)



배우
1987-02-25
남
178cm, 63kg
한국예술종합학교 영상원 연극원 연기과
----------
배우
1968-10-15
남
178cm
----------
소녀시대; girlsgeneration; girls generation
가수
1990-05-30
여
https://www.instagram.com/yoona__lim/
SM엔터테인먼트
----------
1956-08-31
2018-07-23
남
https://twitter.com/hcroh
https://www.facebook.com/omyChans
----------
배우
1996-10-24
여
----------
성우
1946-10-08
여
----------
성우
1952-10-03
남
----------


In [13]:
# 배우 상세 정보 추출을 위한 과정 셀
actors =  soup.select("li.people_li div.name")
actors_detail_info = list() # 배우 상세 정보 저장
for actor in actors:
    # 배우 한명의 상세정보를 위한 url
    #print("http://www.cine21.com" + actor.select_one('a').attrs['href'])
    url_link = "http://www.cine21.com" + actor.select_one('a').attrs['href']

    res = req.get(url_link)
    # 배우 상세정보를 저장하는 soup 객체(html)
    soup_actor = bs(res.content, 'html.parser') 

    # 한 배우에 대한 정보 리스트
    actor_info_dict = dict() # 배우 한명 정보를 담는 dict 변수
    default_info = soup_actor.select_one('ul.default_info')
    # 한 배우 상세정보를 리스트로 리턴받음    
    actor_details = default_info.select('li') 
    for actor_detail in actor_details:
        actor_detail_key = actor_detail.select_one('span.tit').get_text(strip=True)
        #print(type(actor_detail))
        #상세정보 value 추출
        actor_detail_value = re.sub('<span.*?>.*?</span>','', str(actor_detail))
        actor_detail_value = re.sub('<.*?>','', actor_detail_value).strip()
        #배우 한 명의 정보 dict 저장
        #print(actor_detail_value)    
        actor_info_dict[actor_detail_key] = actor_detail_value
        # 방법2 : 상세정보 key와 value 추출 
        # actor_detail_key = actor_detail.get_text() 
        # actor_detail_value = actor_detail.select_one('span').extract().get_text()
        # print(actor_detail_key, ":", actor_detail_value)
    # print("-" * 10) 
    actors_detail_info.append(actor_info_dict)

# 배우들의 상세정보 출력
print(actors_detail_info)




[{'직업': '배우', '생년월일': '1987-02-25', '성별': '남', '신장/체중': '178cm, 63kg', '학교': '한국예술종합학교 영상원 연극원 연기과'}, {'직업': '배우', '생년월일': '1968-10-15', '성별': '남', '신장/체중': '178cm'}, {'다른 이름': '소녀시대; girlsgeneration; girls generation', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': 'https://www.instagram.com/yoona__lim/', '소속사': 'SM엔터테인먼트'}, {'생년월일': '1956-08-31', '사망': '2018-07-23', '성별': '남', '홈페이지': 'https://twitter.com/hcroh\nhttps://www.facebook.com/omyChans'}, {'직업': '배우', '생년월일': '1996-10-24', '성별': '여'}, {'직업': '성우', '생년월일': '1946-10-08', '성별': '여'}, {'직업': '성우', '생년월일': '1952-10-03', '성별': '남'}]


## 배우 흥행 지수 뽑기
- 배우 개인별 흥행지수를 추출 후 ',' 제거하기

In [14]:
# 배우 개인별 흥행지수 추출하기
actors = soup.select('li.people_li div.name')
actors_hit = soup.select('li.people_li ul.num_info strong')
actors_hit_info = []
for idx, actor in enumerate(actors):
    #print("배우이름" ,re.sub('\(\w+\)', '', actor.text))
    actor_hit_num = int(actors_hit[idx].text.replace(',',''))
    #print("흥행지수", actor_hit_num)
    #print('-'*10)    
    actors_hit_info.append(actor_hit_num)
print(actors_hit_info)

[161725, 121294, 80862, 51594, 40431, 27908, 24419]


## 각 배우별 출연 영화를 document에 저장하기
  - 출연 영화는 한 개가 될 수도 있고, 여러 개가 될 수도 있음
  - 파이썬은 리스트, mongodb document는 컬럼에 배열(array)로 넣으면 됨 

In [15]:
# 배우 출현 영화만 추출
movie_list = list()
movies = soup.select('li.people_li ul.mov_list')
#print(movies)
for movie in movies:
    actor_movies = list()
    movie_titles = movie.select('ul.mov_list li a span')
    #print(movie_titles)
    for movie_title in movie_titles:
        actor_movies.append(movie_title.text)
        print(movie_title.text)
    movie_list.append(actor_movies)
    print('-'*10)

기적
----------
기적
----------
기적
----------
노회찬6411
----------
기적
----------
극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들
----------
극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들
----------


### insert_one() 로 하나씩 데이터 입력하기 (반복문과 함께 사용하면, 여러 데이터를 넣을 수 있음)

- actor_list: 배우 이름
- actor_details: 배우 상세 정보
- actor_rate: 흥행 지수
- date: 기준월
- movie_list: 출연 영화 리스트!

In [16]:
# 한 배우 정보(document)씩 몽고DB에 insert
for num, actor in enumerate(actors_list):
    actor_collection.insert_one(
        {"actor":actor, 
         "actor_details": actors_detail_info[num], 
         "actor_hit":actors_hit_info[num], 
         "date":month, 
         "movie_list":movie_list[num]})

### insert_many() 실행

In [ ]:
# dict 타입의 actor들의 정보를 list로 저장
# insert_many()를 수행하기 위해서는 document를 list로 만들어야함.
actors_info = list()   
for num, actor in enumerate(actors_list):
    actors_info.append(
        {"actor":actor, 
         "actor_details": actors_detail_info[num], 
         "actor_rate":actors_hit_info[num], 
         "date":month, 
         "movie_list":movie_list[num]}
    )
print(actors_info) # 리스트 결과 확인
actor_collection.insert_many(actors_info)

### document key 명을 변경하는 방법
- 실습 : actor_details -> actor_info

In [ ]:
#actor_collection.update_many({}, {"$rename": {"원래키" : "변경할 키"} })
actor_collection.update_many({}, {"$rename": {"actor" : "actor_name"} })

In [ ]:
# collection drop
actor_collection.drop()

In [ ]:
# collection 찾기
docs = actor_collection.find()
docs

In [ ]:
for doc in docs:
    print(doc)